# KNeighborsClassifier from scratch

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)  
This work by Jephian Lin is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Algorithm
**Input:**  
- `X`: an array of shape `(N,d)` whose rows are samples and columns are features
- `y`: the labels of shape `(N,)`
- `k`: Numbers of neighbors (including self) to vote
- `algorithm`: `'brute'`, `'ball_tree'`, or `'kd_tree'`

**Output:**  
A tuple `(predict, k_nearest_neighbors)`.  
- `predict`: a function that takes data `X_sample` and output their predicted labels
- `k_nearest_neighbors`: a function that takes data `X_sample` and return an array of shape `(X_sample_height, k)` that stores the indices of the nearest neighbors in `X` for each row in `X_sample`

**Steps:**
1. If `algorithm=="brute"`, create the function `k_nearest_neighbors` by the distance matrix.  
2. If `algorithm=="ball_tree"` or `algorithm=="kd_tree"`, create the function `k_nearest_neighbors` by `sklearn.neighbors.NearestNeighbors` with the corresponding algorithm.
3. Create the function `predict` that executes the following steps:
    1. Input `X_sample` .
    2. Let `nbrhoods = k_nearest_neighbors(X_sample)` .  
    3. Let `votes = y[nbrhoods]` .
    4. Calculate the most frequent label in each row of `votes` and store the results in `y_new` .
    5. Return `y_new` .

## Pseudocode
Translate the algorithm into the pseudocode.  
This helps you to identify the parts that you don't know how to do it.  

    1. Create function knn with inputs: X, y, algorithm, and k
    2. sub-function k_nearest_neighbors return k nearest neighbors of sample X 
    3. sub-function predict return labels of k nearest neighbors
    4. If algorithm=="brute", creating the funciton k_nearest_neighbors by the distance matrix (dist_mtx(X, Y)).
    5. If algorithm=="ball_tree" or algorithm=="kd_tree", create the function k_nearest_neighbors by sklearn.

## Code

In [ ]:
### your answer here
from sklearn.neighbors import NearestNeighbors
from scipy import stats

def dist_mtx(X, Y):
    X_col = X[:,np.newaxis,:]
    Y_row = Y[np.newaxis,:,:]
    diff = X_col - Y_row
    dist = np.linalg.norm(diff, axis=-1)
    return dist

def get_y(kneighbors, y):
    # y[kneighbors] => pick the class value by the index of kneighbors
    # use scipy.stats.mode to find out mode in y, and reshape to vectors
    return stats.mode(y[kneighbors], axis=1).mode.reshape(kneighbors.shape[0])

def knn(X, y, algorithm, k):
    # find out the indeices of k-neighbors
    if algorithm == 'brute':
        def k_nearest_neighbors(X_sample):
            dist = dist_mtx(X_sample, X)
            argp = dist.argpartition(k-1, axis=1)
            return argp[:,:k]
        def predict(X_sample):
            k_neighbors = k_nearest_neighbors(X_sample)
            return get_y(k_neighbors, y)
            
    elif algorithm in ['ball_tree', 'kd_tree']:
        def k_nearest_neighbors(X_sample):
            nbr = NearestNeighbors(n_neighbors=k, algorithm=algorithm)
            nbr.fit(X)
            return nbr.kneighbors(X_sample, return_distance=False)
        def predict(X_sample):
            k_neighbors = k_nearest_neighbors(X_sample)
            return get_y(k_neighbors, y)
    return predict, k_nearest_neighbors

##### Jephian:
Nice.  
It seems that the definition of `predict` is the same in each case, so maybe you may define it outside of the `if` statements.

## Test
Take some sample data from [KNeighborsClassifier-with-scikit-learn](KNeighborsClassifier-with-scikit-learn.ipynb) and check if your code generates similar outputs with the existing packages.

##### Name of the data
Description of the data.

In [ ]:
### results with your code
mu1 = np.array([2.5,0])
cov1 = np.array([[1.1,-1],
                [-1,1.1]])
mu2 = np.array([-2.5,0])
cov2 = np.array([[1.1,1],
                [1,1.1]])
X = np.vstack([np.random.multivariate_normal(mu1, cov1, 100), 
               np.random.multivariate_normal(mu2, cov2, 100)])
y = np.array([0]*100 + [1]*100)
X_sample = np.random.uniform(low=-5, high=5, size=(1000,2))
plt.scatter(*X.T, c=y)

In [ ]:
predict, k_nearest_neighbors  = knn(X, y, 'brute', 2)
y_new = predict(X_sample)
plt.figure(figsize=(6, 6)) 
plt.subplot(311).set_title('brute')
plt.scatter(*X_sample.T, c=y_new)

predict, k_nearest_neighbors  = knn(X, y, 'ball_tree', 2)
y_new = predict(X_sample)
plt.subplot(312).set_title('ball_tree')
plt.scatter(*X_sample.T, c=y_new)

predict, k_nearest_neighbors  = knn(X, y, 'kd_tree', 2)
y_new = predict(X_sample)
plt.subplot(313).set_title('kd_tree')
plt.scatter(*X_sample.T, c=y_new)

In [ ]:
### results with existing packages
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X, y)
y_new = model.predict(X_sample)
plt.figure(figsize=(6, 2))
plt.suptitle('sklearn')
plt.scatter(*X_sample.T, c=y_new)

## Comparison

##### Exercise 1
Let  
```python
t = np.arange(20)
angle = 2 * np.pi / 20 * t
X1 = np.vstack([np.cos(angle), np.sin(angle)]).T
X2 = 5 * X1
X = np.vstack([X1, X2])
y = np.array([0]*20 + [1]*20)
X_sample = 10 * np.random.rand(1000,2) - np.array([5,5])
```

###### 1(a)
Train a $k$-nearest neighbors classification model by `X` and `y` .  
Make a prediction of `X_sample` by:  
1. your code with different algorithm settings
2. `sklearn.neighbors.KNeighborsClassifier`

The results should be the same.  
Check if this is true.

In [ ]:
### your answer here
t = np.arange(20)
angle = 2 * np.pi / 20 * t
X1 = np.vstack([np.cos(angle), np.sin(angle)]).T
X2 = 5 * X1
X = np.vstack([X1, X2])
y = np.array([0]*20 + [1]*20)
X_sample = 10 * np.random.rand(1000,2) - np.array([5,5])

predict, k_nearest_neighbors = knn(X, y, 'brute', 2)
kneighbors_scratch_brute = k_nearest_neighbors(X_sample)
y_new_scratch_brute = predict(X_sample)

predict, k_nearest_neighbors = knn(X, y, 'ball_tree', 2)
kneighbors_scratch_ball_tree = k_nearest_neighbors(X_sample)
y_new_scratch_ball_tree = predict(X_sample)

predict, k_nearest_neighbors = knn(X, y, 'kd_tree', 2)
kneighbors_scratch_kd_tree = k_nearest_neighbors(X_sample)
y_new_scratch_kd_tree = predict(X_sample)

model = KNeighborsClassifier(n_neighbors=2)
model.fit(X, y)
y_new_scikit = model.predict(X_sample)

print(((y_new_scratch_brute==y_new_scratch_ball_tree) & 
       (y_new_scratch_ball_tree==y_new_scratch_kd_tree) & 
       (y_new_scratch_kd_tree==y_new_scikit)).all())

###### 1(b)
Let `y_new` be the prediction of `X_sample` in the previous question. 
Plot the points (rows) in `X` with `c=y` .  
Plot the points (rows) in `X_sample` with `c=y_new` and `alpha=0.1` .

In [ ]:
### your answer here
plt.scatter(*X.T, c=y, label="Training data", marker="^", s=60)
plt.scatter(*X_sample.T, c=y_new_scratch_brute, alpha=0.1, label="prediction")
plt.legend(loc="upper left")

###### 1(c)
Let  
```python
model = KNeighborsClassifier()
model.fit(X, y)
```  
and let `k_nearest_neighbors` be one of the output of your function.  
The results of `k_nearest_neighbors(X_sample)` should be the same as `model.kneighbors(X_sample, return_distance=False)` .  
(The corresponding rows contains the same collection of elements, but might be in different order.)  
Check if this is true.

In [ ]:
### your answer here
predict, k_nearest_neighbors = knn(X, y, 'brute', 2)
neighbors = k_nearest_neighbors(X_sample)
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X, y)
print((neighbors==model.kneighbors(X_sample, return_distance=False)).all())

##### Exercise 2
Let  
```python
m,n = 8,8
frames = (m-2) * (n-2)

o = np.array([[1,1,1],
              [1,0,1],
              [1,1,1]])
x = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]])
oo = np.zeros((frames, m, n))
xx = np.zeros((frames, m, n))
count =  0
for i in range(m-2):
    for j in range(n-2):
        oo[count, i:i+3, j:j+3] = o
        xx[count, i:i+3, j:j+3] = x
        count += 1


X = np.vstack([oo, xx]).reshape(2*frames, -1)
y = np.array([0]*frames + [1]*frames)
```

###### 2(a)
Run  
```python
plt.imshow(oo[i], cmap="Greys")
```
with different `i` .  
Guess what is the meaning of `oo` and `xx` .

In [ ]:
### your answer here
m,n = 8,8
frames = (m-2) * (n-2)

o = np.array([[1,1,1],
              [1,0,1],
              [1,1,1]])
x = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]])
oo = np.zeros((frames, m, n))
xx = np.zeros((frames, m, n))
count =  0
for i in range(m-2):
    for j in range(n-2):
        oo[count, i:i+3, j:j+3] = o
        xx[count, i:i+3, j:j+3] = x
        count += 1

X = np.vstack([oo, xx]).reshape(2*frames, -1)
y = np.array([0]*frames + [1]*frames)

In [ ]:
for i in range(6):
    for j in range(6):
        ax = plt.subplot2grid((6,6), (i,j))
        ax.imshow(oo[i*6+j], cmap="Greys")
print('oo is a 36 images set, an o-like object in each image from left to right, up to down with step 1')
print('xx is a 36 images set, an x-like object in each image from left to right, up to down with step 1')

###### 2(b)
Train a $k$-nearest neighbors classification model by `X` an `y` .  
Make a prediction `y_new` for the training data `X` .  
What is the outcome?  
Can you give a reason to this phenomenon?

In [ ]:
### your answer here
for k in range(1,10):
    predict, k_nearest_neighbors = knn(X, y, 'brute', k)
    y_new = predict(X)
    print(y_new)


#### Reason
```
when test data is o[0], then
   || o[0]  - o[0] || = 0 
   || o[0]  - o[1] || = 5  
   || o[0]  - o[2] || = 10 
   || o[0]  - o[3~5] || = 9 
   || o[0]  - o[6] || = ...
   || o[0]  - x[0] || = 5  
   || o[0]  - x[1] || = 7  
   || o[0]  - x[2] || = 9  
   || o[0]  - x[3~5] || = 13
   and so on ...

else if test data is x[0], then
   || x[0]  - o[0] || = 5 
   and so on ...

Thus, 
when k=1, the one itself will be the only answer in all 72 images, accuracy = 100%
when k=2, ...
and so on ...

Different k will got different nearest neighbors (lowest norm between) from o and x, and result in different inference
```

##### Jephian:
Indeed, so the brief answer is that the neighbors of a point in `oo` are mostly points in `xx` but not points in `oo` .

##### Jephian:
Well done!